In [1]:
'''main.ipynb is used for running all simulations and includes necessary parameters only.
    Inputs
    Surface --> domain shape/size
    Bacteria Surface --> seed, surface, concentration, domain shape/size, width and length (size_x and size_y)
    Outputs (in Excel File)
    Seed
    '''
import import_ipynb #allows import of other jupyter notebooks
from openpyxl import Workbook, load_workbook #allows access to excel program
from openpyxl.utils.dataframe import dataframe_to_rows #access to each sheets in the excel file
from openpyxl.utils import get_column_letter #allows access to letters of each column
import pickle #used to save object (data persistence)
import numpy as np
import os #access to operating system but our purposes are for directory access
import timeit #time it takes for code to execute

dirname = r"" #find directory of this file
wb = Workbook() #creates excel file
ws1 = wb.create_sheet("Results",0)
#naming the columns in the worksheet
ws1.cell(1,1, "Surface Characteristics:")
ws1.cell(1,2, "Bacteria Characteristics:")
ws1.cell(1,3, "Seed # ")
ws1.cell(1,4, "Min Energy:")
ws1.cell(1,5, "Min X:")
ws1.cell(1,6, "Min Y:")
ws1.cell(1,7, "Surface Charge at Min Energy:")
ws1.cell(1,8, "Min Energy Gradient Strip: ")
ws1.cell(1,9, "Histogram: ")
#create numbering for histogram plot
count = 0
for i in range(10, 31):
    ws1.cell(1,i, count)
    ws1.cell(2, i, 0)
    count += 1

#adjust column width to text length
for i in range(ws1.max_column):
    text = ws1.cell(1, i+1).value
    if type(text) == int:
        text = str(text)
        column_width = len(text) + 1
    else:
        column_width = len(text)
    ws1.column_dimensions[get_column_letter(i+1)].width = column_width

print("The following prompt will ask the type of simulation you want to run, \n input 1 for single simulation, \n input 2 for multiple simulations with same surface and different bacteria, \n input 3 for multiple simulations with same bacteria and different surface")
num_sim = input("Type of simulation: ")
num_sim = int(num_sim)

if num_sim == 1:
    #running 1 simulation
    #asks for user-input
    print("Calibrating Surface...input values")
    surf_seed = input("Enter seed number (>1): ")
    surf_seed = int(surf_seed)

    domain = input("Enter domain shape, Choose from (single, cross, small, medium, large, xlarge, xxlarge, xxxlarge, \
x4large, x5large, x6large, x7large, x8large, x9large, x10large, x11large, x12large, x13large, x14large, x15large, x16large): ")
    #create surface
    if domain == "x8large" or domain == "x9large" or domain == "x7large" or domain == "x6large" or domain == "x5large":
        from main_surfacev4_x5_x9_neg import Net_Zero_Surface
    elif domain == "x10large" or domain == "x11large":
        from main_surfacev4_x10_x11_neg import Net_Zero_Surface
    elif domain == "x12large" or domain == "x13large":
        from main_surfacev4_x12_x13_neg import Net_Zero_Surface
    elif domain == "x14large" or domain == "x15large":
        from main_surfacev4_x14_x15_neg import Net_Zero_Surface
    elif domain == "x16large":
        from main_surfacev4_x16_neg import Net_Zero_Surface
    else:
        from main_surfacev4_single_x4_neg import Net_Zero_Surface
    main_surface = Net_Zero_Surface(domain, surf_seed) 

    #ask for user-input
    print("Calibrating Bacteria...input values")
    seed = input("Enter seed number(>1): ")
    seed = int(seed)
    conc = input("Enter concentration of positive domains (0 - 1): ")
    conc = float(conc)

    b_domain = input("Enter domain shape, Choose from (single, cross, small, medium, large, xlarge, xxlarge, xxxlarge, \
x4large, x5large, x6large, x7large, x8large, x9large, x10large, x11large, x12large, x13large, x14large, x15large, x16large): ")
    
    from bacteria_surfacev4 import Bacteria
        
    size_x = input("Enter number of points in x-axis (50-500): ")
    size_x = int(size_x)
    size_y = input("Enter number of points in y-axis (50-500): ")
    size_y = int(size_y)
    #create bacteria surface
    bact_surface = Bacteria(seed, main_surface, conc, b_domain, size_x, size_y)

    interval_x = input("Enter size of interval in x-direction (1-100): ")
    interval_x = int(interval_x)
    interval_y = input("Enter size of interval in y-direction (1-100): ")
    interval_y = int(interval_y)
    min_energy, min_x, min_y, min_energy_charge, min_charge, min_charge_x, min_charge_y = bact_surface.interact(interval_x, interval_y)

    grad_strip = min_x // 500

    ws1.cell(seed, 1, domain)
    ws1.cell(seed, 2, b_domain)
    ws1.cell(seed, 3, seed)
    ws1.cell(seed, 4, min_energy)
    ws1.cell(seed, 5, min_x)
    ws1.cell(seed, 6, min_x)
    ws1.cell(seed, 7, min_energy_charge)
    ws1.cell(seed, 8, grad_strip)
    
elif num_sim == 2:

    #running multiple simulations: same surface, different bacteria (seed)
    #asks for user-input
    print("Calibrating Surface...input values")
    surf_seed = input("Enter seed number(>1): ")
    surf_seed = int(surf_seed)
    domain = input("Enter domain shape, Choose from (single, cross, small, medium, large, xlarge, xxlarge, xxxlarge, \
x4large, x5large, x6large, x7large, x8large, x9large, x10large, x11large, x12large, x13large, x14large, x15large, x16large): ")
    if domain == "x8large" or domain == "x9large" or domain == "x7large" or domain == "x6large" or domain == "x5large":
        from main_surfacev4_x5_x9_neg import Net_Zero_Surface
    elif domain == "x10large" or domain == "x11large":
        from main_surfacev4_x10_x11_neg import Net_Zero_Surface
    elif domain == "x12large" or domain == "x13large":
        from main_surfacev4_x12_x13_neg import Net_Zero_Surface
    elif domain == "x14large" or domain == "x15large":
        from main_surfacev4_x14_x15_neg import Net_Zero_Surface
    elif domain == "x16large":
        from main_surfacev4_x16_neg import Net_Zero_Surface
    else:
        from main_surfacev4_single_x4_neg import Net_Zero_Surface
    print("Creating Surface...")
    main_surface = Net_Zero_Surface(domain, surf_seed)
    #create surface

    #ask for user-input
    print("Calibrating Bacteria...input values")
    print("Note: seed start and end are the number of simulations you want to run")
    seed_start = input("Enter seed start number(>1): ")
    seed_start = int(seed_start)
    seed_end = input("Enter seed end number(>2): ")
    seed_end = int(seed_end)
    conc = input("Enter concentration of positive domains(0-1): ")
    conc = float(conc)
    b_domain = input("Enter domain shape, Choose from (single, cross, small, medium, large, xlarge, xxlarge, xxxlarge, \
x4large, x5large, x6large, x7large, x8large, x9large, x10large, x11large, x12large, x13large, x14large, x15large, x16large): ")
    from main_bacteriav4 import Bacteria
    size_x = input("Enter number of points in x-axis (50-500): ")
    size_x = int(size_x)
    size_y = input("Enter number of points in y-axis (50-500): ")
    size_y = int(size_y)
    print("Calibrating Bacteria Interactions...input values")
    interval_x = input("Enter size of interval in x-direction (1-100): ")
    interval_x = int(interval_x)
    interval_y = input("Enter size of interval in y-direction (1-100): ")
    interval_y = int(interval_y)
    #create bacteria surface
    counter = 0
    start_time = timeit.default_timer()
    for seed in range(seed_start, seed_end):
        bact_surface = Bacteria(seed, main_surface, conc, b_domain, size_x, size_y)

        min_energy, min_x, min_y, min_energy_charge, min_charge, min_charge_x, min_charge_y = bact_surface.interact(interval_x, interval_y)
        print("done: " + str(counter))
        counter += 1
        grad_strip = min_x // 500
        ws1.cell(seed, 1, domain)
        ws1.cell(seed, 2, b_domain)
        ws1.cell(seed, 3, seed)
        ws1.cell(seed, 4, min_energy)
        ws1.cell(seed, 5, min_x)
        ws1.cell(seed, 6, min_x)
        ws1.cell(seed, 7, min_energy_charge)
        ws1.cell(seed, 8, grad_strip)
    elapsed = timeit.default_timer() - start_time #stop timer
    elapsed_min = elapsed / 60 #convert second to minutes
    ws1.cell(4, 9, elapsed_min) #save time elapsed in Excel sheet

elif num_sim == 3:
    #running multiple simulations: same bacteria, different surface (seed)
    #ask for user-input
    print("Calibrating Bacteria...input values")
    seed = input("Enter seed number: ")
    seed = int(seed)
    conc = input("Enter concentration of positive domains (0 - 1): ")
    conc = float(conc)

    b_domain = input("Enter domain shape, Choose from (single, cross, small, medium, large, xlarge, xxlarge, xxxlarge, \
x4large, x5large, x6large, x7large, x8large, x9large, x10large, x11large, x12large, x13large, x14large, x15large, x16large): ")
    from main_bacteriav4_b import Bacteria
    size_x = input("Enter number of points in x-axis (50-500): ")
    size_x = int(size_x)
    size_y = input("Enter number of points in y-axis (50-500): ")
    size_y = int(size_y)
    #create bacteria surface
    bact_surface = Bacteria(seed, conc, b_domain, size_x, size_y)
    print("Calibrating Bacteria Interactions...input values")
    interval_x = input("Enter size of interval in x-direction (1-100): ")
    interval_x = int(interval_x)
    interval_y = input("Enter size of interval in y-direction (1-100): ")
    interval_y = int(interval_y)
    
    #asks for user-input
    print("Calibrating Surface...input values")
    print("Note: seed start and end are the number of simulations you want to run")
    seed_start = input("Enter start seed number(>1): ")
    seed_start = int(seed_start)
    seed_end = input("Enter end seed number(>2): ")
    seed_end = int(seed_end)
    domain = input("Enter domain shape, Choose from (single, cross, small, medium, large, xlarge, xxlarge, xxxlarge, \
x4large, x5large, x6large, x7large, x8large, x9large, x10large, x11large, x12large, x13large, x14large, x15large, x16large): ")
    if domain == "x8large" or domain == "x9large" or domain == "x7large" or domain == "x6large" or domain == "x5large":
        from main_surfacev4_x5_x9_neg import Net_Zero_Surface
    elif domain == "x10large" or domain == "x11large":
        from main_surfacev4_x10_x11_neg import Net_Zero_Surface
    elif domain == "x12large" or domain == "x13large":
        from main_surfacev4_x12_x13_neg import Net_Zero_Surface
    elif domain == "x14large" or domain == "x15large":
        from main_surfacev4_x14_x15_neg import Net_Zero_Surface
    elif domain == "x16large":
        from main_surfacev4_x16_neg import Net_Zero_Surface
    else:
        from main_surfacev4_single_x4_neg import Net_Zero_Surface
    #create surface
    counter = 0
    start_time = timeit.default_timer() #start timer
    for surf_seed in range(seed_start, seed_end):
        main_surface = Net_Zero_Surface(domain, surf_seed) 
        min_energy, min_x, min_y, min_energy_charge, min_charge, min_charge_x, min_charge_y = bact_surface.interact(main_surface, interval_x, interval_y)
        print("done: " + str(counter))
        counter += 1
        grad_strip = min_x // 500

        ws1.cell(surf_seed, 1, domain)
        ws1.cell(surf_seed, 2, b_domain)
        ws1.cell(surf_seed, 3, seed)
        ws1.cell(surf_seed, 4, min_energy)
        ws1.cell(surf_seed, 5, min_x)
        ws1.cell(surf_seed, 6, min_x)
        ws1.cell(surf_seed, 7, min_energy_charge)
        ws1.cell(surf_seed, 8, grad_strip)
    elapsed = timeit.default_timer() - start_time #stop timer
    elapsed_min = elapsed / 60 #convert second to minutes
    ws1.cell(4, 9, elapsed_min) #save time elapsed in Excel sheet

#count number of min_energy locations at each gradient strip
if num_sim == 2:
    for row in range(seed_start, seed_end):
        val_id = ws1.cell(row, 8).value
        val = ws1.cell(2, 10+int(val_id)).value
        ws1.cell(2, 10+int(val_id), int(val)+1)
    
#saves excel file in same directory as this file
if num_sim == 2:
    name = "zero_" + "surf_" + str(surf_seed) + "_interval_" + str(interval_x) + "_b_" + str(b_domain) + "_" + str(conc) + "_s_" + str(domain) + "seed_" + str(seed_start) + "-" + str(seed_end) #change name of file
elif num_sim == 3:
    name = "zero_" + "surf_" + str(seed_start) + "-" + str(seed_end) + "_interval_" + str(interval_x) + "_b_" + str(b_domain) + "_" + str(conc) + "_s_" + str(domain) + "seed_" + str(seed) #change name of file
wb.save(dirname + "\\{}.xlsx".format(name))
print("COMPLETED")

The following prompt will ask the type of simulation you want to run, 
 input 1 for single simulation, 
 input 2 for multiple simulations with same surface and different bacteria, 
 input 3 for multiple simulations with same bacteria and different surface
Type of simulation: 3
Calibrating Bacteria...input values
Enter seed number: 2
Enter concentration of positive domains (0 - 1): 0.2
Enter domain shape, Choose from (single, cross, small, medium, large, xlarge, xxlarge, xxxlarge, x4large, x5large, x6large, x7large, x8large, x9large, x10large, x11large, x12large, x13large, x14large, x15large, x16large): x7large
importing Jupyter notebook from main_bacteriav4_b.ipynb
Enter number of points in x-axis (50-500): 100
Enter number of points in y-axis (50-500): 100
Calibrating Bacteria Interactions...input values
Enter size of interval in x-direction (1-100): 10
Enter size of interval in y-direction (1-100): 10
Calibrating Surface...input values
Note: seed start and end are the number of simul